In [1]:
CSV_IN = 'datos/Base datos_INTA_ascii.csv'
CSV_OUT = CSV_IN.replace('.csv', '_limpio.csv')
def guardar():
    df.to_csv(CSV_OUT, index=None)

In [2]:
print 'Cargando datos de "%s"' % CSV_IN
df = pd.read_csv(CSV_IN)
print 'Cargado dataframe de tamaño %d x %d' % df.shape
# df.head()

Cargando datos de "datos/Base datos_INTA_ascii.csv"
Cargado dataframe de tamaño 423 x 44


In [3]:
cols = list(df.columns)
for c in cols:
    if c.startswith('Unnamed'):
        df = df.drop(c, axis=1)
guardar()
# for c in df.columns: print c

In [4]:
# Convertir los valores de Campana de "0708" a 2007
def convertir_campana(num):
    if pd.isnull(num):
        return np.nan
    return '%d-%d' % (2000 + int(num / 100), 2000 + int(num % 100))

df['Campana'] = df['Campana'].apply(convertir_campana)
guardar()

In [5]:
# Convertir los nombres de columnos a minisculas
def limpiar_nombre_de_columno(col):
    col = col.lower().replace('%', 'pct')
    col = ''.join([ch if (ch.isalnum() or ch == '_') else '_' for ch in col])
    while '__' in col:
        col = col.replace('__', '_')
    col = col.strip('_').strip()
    return col

df.columns = [limpiar_nombre_de_columno(c) for c in df.columns]
guardar()
# for c in df.columns: print c

In [6]:
columnos_innecesarios = ['empresa', 'establecimiento', 'lote']
# ******************************************************
# ******************************************************
# ********** BUEN EJEMPLO ******************************
# ******************************************************
# ******************************************************
columnos_con_demasiado_na = ['fert_cober_clk_fecha']
df = df.drop(columnos_innecesarios + columnos_con_demasiado_na, axis=1)
guardar()

In [7]:
positivos = ['fenologia_emerg_cosecha_dias',
             'rinde_con_factor_kg_ha',
             'superficie_sembrada_has']
#              'fert_cober_preriego_cant_n_kg_ha']
for c in positivos:
    invalidos = df[c] <= 0
    n = invalidos.sum()
    print 'Cambiando %d (%.0f%%) valores invalidas de "%s" a NA' % (
        n, 100 * n / len(df), c)
    df[c][invalidos] = np.nan
guardar()

Cambiando 38 (8%) valores invalidas de "fenologia_emerg_cosecha_dias" a NA
Cambiando 38 (8%) valores invalidas de "rinde_con_factor_kg_ha" a NA
Cambiando 0 (0%) valores invalidas de "superficie_sembrada_has" a NA


/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [8]:
# Algunos porcentajes fueron entrados commo decimal,
# por ejemplo 0.20 en ves de 20 para 20%
# ******************************************************
# ******************************************************
# ********** BUEN EJEMPLO ******************************
# ******************************************************
# ******************************************************
df['factor_rend_pcthcosecha'] = df['factor_rend_pcthcosecha'].apply(
    lambda pct: pct * 100 if pct < 1 else pct)
guardar()

In [9]:
df['sistema_laboreo'] = df['sistema_laboreo'].apply(lambda s: s.strip())
guardar()

In [10]:
# ******************************************************
# ******************************************************
# ********** BUEN EJEMPLO ******************************
# ******************************************************
# ******************************************************
print df['cultivo_antecesor'].value_counts().sort_index()

def lower(s):
    try:
        return s.lower()
    except AttributeError:
        return s
for col, dtype in df.dtypes.iteritems():
    if dtype == 'O':
        df[col] = df[col].apply(lower)

print
print df['cultivo_antecesor'].value_counts().sort_index()
guardar()

ARROZ1           6
ARROZ2          19
ARROZ3          16
Arroz1          58
Arroz2          28
Arroz3          12
Arroz4o+        18
Campo tural     39
Desc1          114
Desc2           29
Desc3           14
Desc4o+          2
Maiz             2
RGA             32
Soja            34
dtype: int64

arroz1          64
arroz2          47
arroz3          28
arroz4o+        18
campo tural     39
desc1          114
desc2           29
desc3           14
desc4o+          2
maiz             2
rga             32
soja            34
dtype: int64


In [11]:
# Esta bien los valores de 0 para la candidad de N/P/K?

In [12]:
df.head()

,id,campana,sector,superficie_sembrada_has,superficie_cosechada_has,cultivo_antecesor,variedad,sistema_laboreo,fenologia_siembra_fecha,densidad_siembra_kg_ha,fenologia_emergencia_70pct_fecha,factor_rend_densidad_pl_m2,analisis_suelo_n,analisis_suelo_p,analisis_suelo_k,aporte_elemento_n_kg_ha,aporte_elemento_p_kg_ha,aporte_elemento_k_kg_ha,fert_cober_preriego_fecha,fert_cober_preriego_formulado,fert_cober_preriego_cant_n_kg_ha,fert_cober_dpf_fecha,fert_cober_dpf_cant_n_kg_ha,fert_cober_clk_cant_kg_ha,fenologia_dpf_fecha,fenologia_emerg_dpf_dias,riego_inicio_fecha,riego_chacra_completa_fecha,riego_velocidad_dias,riego_emerg_lote_completo_dias,fenologia_flor_50pct_fecha,fenologia_emerg_flor_50pct_dias,riego_finalizacion_fecha,fenologia_cosecha_fecha,fenologia_emerg_cosecha_dias,factor_rend_pcthcosecha,rendimiento_seco_kg_ha,factor_rend_pctfactor,rinde_con_factor_kg_ha
0,"0708irundymirungaherrerito 27877,25",2007-2008,herrerito,8.0,8.0,campo tural,irga417,conv,2007-10-25 00:00:00,103,2007-11-07 00:00:00,NaN,8.5,22.236,28.22,8.5,22.236,28.22,2007-12-04 00:00:00,80,36.8,NaN,NaN,NaN,NaN,NaN,2007-12-03 00:00:00,2007-12-05 00:00:00,2,28,2008-02-06 00:00:00,91,2008-03-03 00:00:00,2008-03-22 00:00:00,136,21.0,7877.250000,1.0521,8287.654725
1,"0708irundymirungaherrerito 38597,1914893617",2007-2008,herrerito,23.5,23.5,campo tural,taim,conv,2007-10-25 00:00:00,103,2007-11-07 00:00:00,NaN,8.5,22.236,28.22,8.5,22.236,28.22,2007-12-04 00:00:00,80,36.8,NaN,NaN,NaN,NaN,NaN,2007-12-03 00:00:00,2007-12-07 00:00:00,4,30,2008-02-06 00:00:00,91,2008-03-03 00:00:00,2008-03-17 00:00:00,131,22.5,8597.191489,1.0650,9156.008936
2,"0708irundymirungaherrerito 48503,07462686567",2007-2008,herrerito,33.5,33.5,campo tural,taim,conv,2007-10-25 00:00:00,103,2007-11-07 00:00:00,NaN,8.5,22.236,28.22,8.5,22.236,28.22,2007-12-04 00:00:00,80,36.8,NaN,NaN,NaN,NaN,NaN,2007-12-03 00:00:00,2007-12-09 00:00:00,6,32,2008-02-07 00:00:00,92,2008-03-03 00:00:00,2008-03-17 00:00:00,131,18.0,8503.074627,0.9230,7848.337881
3,"0708irundymirungaherrerito 57815,1871657754",2007-2008,herrerito,18.7,18.7,campo tural,irga417,conv,2007-10-26 00:00:00,103,2007-11-06 00:00:00,NaN,8.5,22.236,28.22,8.5,22.236,28.22,2007-12-12 00:00:00,80,36.8,NaN,NaN,NaN,NaN,NaN,2007-12-03 00:00:00,2007-12-06 00:00:00,3,30,2008-02-06 00:00:00,92,2008-03-03 00:00:00,2008-03-23 00:00:00,138,19.0,7815.187166,1.0740,8393.511016
4,"0708irundymirungaherrerito 68945,95238095238",2007-2008,herrerito,37.8,37.8,campo tural,taim,conv,2007-10-25 00:00:00,103,2007-11-06 00:00:00,NaN,8.5,22.236,28.22,8.5,22.236,28.22,2007-12-12 00:00:00,80,36.8,NaN,NaN,NaN,NaN,NaN,2007-12-03 00:00:00,2007-12-09 00:00:00,6,33,2008-02-08 00:00:00,94,2008-03-03 00:00:00,2008-03-24 00:00:00,139,19.0,8945.952381,0.9105,8145.289643


In [13]:
df['superficie_sembrada_has'].order()

13       0.90
172      5.00
160      5.00
337      5.20
316      5.20
363      5.50
300      5.50
281      5.50
147      6.00
130      6.00
6        7.00
15       7.90
0        8.00
20       8.10
16       8.80
21       9.20
33       9.90
18      11.60
36      12.50
31      12.50
59      12.60
17      13.30
14      13.60
25      13.60
48      13.70
34      14.30
32      14.30
11      14.56
108     14.70
29      15.90
        ...  
417     63.00
185     63.00
180     63.20
168     63.20
353     64.40
293     64.40
274     64.40
290     64.40
317     65.10
328     65.10
267     66.20
258     67.30
262     67.80
271     67.80
412     73.40
117     73.40
200     76.00
189     78.90
395     78.90
86      83.60
277     87.50
123     87.70
389     87.70
120     91.50
88      91.50
295     95.20
276     95.20
121    120.60
388    123.00
122    123.00
Name: superficie_sembrada_has, dtype: float64